In [2]:
#Topics covered
#Memory references
#What variables really are 
# memory management
# reference counting and garbage collection
#Dynamic and static typing
#Mutability and immutability
# Shared references!
#Variable equality
#Everything is an object including Classes themselves!

In [3]:
#Variables and Memory references!

In [5]:
# Variable are Memory references!
#Check more on heap memory!
# Storing and retrieving objects from heap is taken care of 
#by the python memory manager!

In [7]:
my_var = 10
#creates an object in memory  at address say 1000 and stores it there!

In [8]:
# my_var is a reference to 10, it is a pointer aka memory address of 10